# Imports

In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append('/home/hice1/mbibars3/scratch/vlm-debiasing/VLM-Debiasing-Project/scripts')

In [33]:
%autoreload 2
import model as m
import torch
import torch.nn as nn
import torch.optim as optim


In [31]:
# Assuming MultiModalPerceiver is defined as in the code above
# Define input dimensions for each modality
input_dims = [64, 128, 256]  # These could be the feature dimensions for each modality

# Initialize MultiModalPerceiver model
model = m.MultiModalPerceiver(
    input_dims=input_dims,
    input_channels=1,
    input_axis=1,
    projection_dim=256,
    num_latents=16,
    latent_dim=128,
    depth=8,
    cross_heads=8,
    latent_heads=8,
    cross_dim_head=32,
    latent_dim_head=32,
    attn_dropout=0.1,
    ff_dropout=0.0,
    output_dim=1,
    weight_tie_layers=True,
    fourier_encode_data=False,
    max_freq=10,
    num_freq_bands=4
)

# Create random data for each modality
batch_size = 10  # Number of samples in a batch
modality_1 = torch.randn(batch_size, input_dims[0])  # First modality with input dimension 64
modality_2 = torch.randn(batch_size, input_dims[1])  # Second modality with input dimension 128
modality_3 = torch.randn(batch_size, input_dims[2])  # Third modality with input dimension 256

# Combine modalities into a list and pass through the model
input_data = [modality_1, modality_2, modality_3]
output = model(input_data)

# Print output shape to verify
print("Output shape:", output.shape)  # Expected shape: [batch_size, output_dim]


torch.Size([10, 768, 1])
Output shape: torch.Size([10, 1])


In [32]:
print(output)

tensor([[-1.0926],
        [-1.1314],
        [-1.0889],
        [-1.1205],
        [-1.0704],
        [-1.0984],
        [-1.0807],
        [-1.0941],
        [-1.1047],
        [-1.1045]], grad_fn=<AddmmBackward0>)


In [34]:
# Dummy target output (regression target)
target = torch.randn(batch_size, 1)  # Shape: [batch_size, output_dim]

learning_rate = 0.001
num_epochs = 5
# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    inputs = [modality_1, modality_2, modality_3]
    output = model(inputs)
    
    # Compute loss
    loss = criterion(output, target)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    # Print loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 2.0485
Epoch [2/5], Loss: 16.5415
Epoch [3/5], Loss: 3.2676
Epoch [4/5], Loss: 1.2193
Epoch [5/5], Loss: 2.2094
